In [1]:
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict

## Global parameters

In [2]:
binary_relevance = [0,1]
cut_off = 3

## Step 1: Simulate Rankings of Relevance for E and P

In [3]:
def identify_docs(docs1, docs2):  
  """
  Take two lists of document relevance scores and generate all possible combinations of overlap
  between the two lists.
  :param docs1: ranking 1 relevance scores
  :param docs2: ranking 2 relevance scores
  :return: list of lists of tuples with all possible overlapping schemes
  """
  
  # Generate all possible document identifiers for irrelevant documents
  zero_ids1 = list(range(docs1.count(0))) 
  if len(zero_ids1) == 0:
    zero_ids2 = [[-1] * docs2.count(0)]  
  else:
    zero_ids2 = list(itertools.permutations(list(range(6)), docs2.count(0)))
    zero_ids2 = sorted([[-1 if x>=docs1.count(0) else x for x in ids] for ids in zero_ids2])
    zero_ids2 = list(ids for ids,_ in itertools.groupby(zero_ids2))  
  
  # Generate all possible document identifiers for relevant documents
  one_ids1 = list(range(docs1.count(1))) 
  if len(one_ids1) == 0:
    one_ids2 = [[-1] * docs2.count(1)]
  else:
    one_ids2 = list(itertools.permutations(list(range(6)), docs2.count(1)))
    one_ids2 = sorted([[-1 if x>=docs1.count(1) else x for x in ids] for ids in one_ids2])
    one_ids2 = list(ids for ids, _ in itertools.groupby(one_ids2))

  # Label the documents of ranking 1
  ranking1 = []
  zero_count = one_count = 0
  for doc in docs1:
    if doc == 0:
      ranking1.append((doc, zero_ids1[zero_count]))
      zero_count += 1
    else:
      ranking1.append((doc, one_ids1[one_count]))
      one_count += 1
      
  # Label the documents of ranking 2
  labelled_rankings = []
  for zero_ids in zero_ids2:
    for one_ids in one_ids2:
      ranking2 = []
      zero_count = one_count = 0
      for doc in docs2:
        if doc == 0:
          ranking2.append((doc, zero_ids[zero_count]))
          zero_count += 1
        else:
          ranking2.append((doc, one_ids[one_count]))
          one_count += 1        
      labelled_rankings.append([ranking1, ranking2])

  return labelled_rankings

In [4]:
system_e = list(map(list, itertools.product(binary_relevance, repeat=cut_off)))
system_p = list(map(list, itertools.product(binary_relevance, repeat=cut_off)))

ranking_pairs = [list(ranking) for ranking in list(itertools.product(system_e, system_p))]
labelled_rankings = [ranking for docs in ranking_pairs for ranking in identify_docs(docs[0], docs[1])]
labelled_rankings

[[[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, -1), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, -1), (0, 0)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, -1), (0, 1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, -1), (0, 2)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 0), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 0), (0, 1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 0), (0, 2)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 1), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 1), (0, 0)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 1), (0, 2)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 2), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 2), (0, 0)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, -1), (0, 2), (0, 1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, 0), (0, -1), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, 0), (0, -1), (0, 1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, 0), (0, -1), (0, 2)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, 0), (0, 1), (0, -1)]],
 [[(0, 0), (0, 1), (0, 2)], [(0, 0), (0

## Defining Expected Reciprocal Rank @ Cut-off (ERR@-)

In [97]:
mapping_relevance_to_probability = lambda pos_g, max_g: ((2**pos_g) - 1 ) / (2**max_g)

In [98]:
def ERR(ranking, mapping=mapping_relevance_to_probability, n=cut_off):
    p, err = 1, 0
    
    for r in range(0, n):
        R = mapping_relevance_to_probability(ranking[r], 1)
        err += (p * (R/(r+1)))
        p *= (1-R)
        
    return err

## Construct the table

In [4]:
table = defaultdict(lambda:list())
for pair in ranking_pairs:
    
    err_e = ERR(pair[0])
    err_p = ERR(pair[1])
    delta_err = err_e - err_p
    if delta_err >= 0: # only consider cases where system e outperforms system p
        table[delta_err].append(pair)
    
table
# table = {k:sum(v) for k,v in table.items()}
    
# table = pd.DataFrame(table, index=[0]).T
# table.reset_index(level=0, inplace=True)

NameError: name 'ERR' is not defined

In [135]:
bins = np.linspace(0.05, 0.95 , 10)
groups = table.groupby(pandas.cut(table['index'], bins))
# groups.count()[0]

index
(0.05, 0.15]    54
(0.15, 0.25]    36
(0.25, 0.35]    31
(0.35, 0.45]    14
(0.45, 0.55]     9
(0.55, 0.65]     3
(0.65, 0.75]     7
(0.75, 0.85]     8
(0.85, 0.95]     1
Name: 0, dtype: int64

In [ ]:
def team_draft_interleaving(ranking_a, ranking_b):
    I, team_a, team_b = list(), set(), set()
    